In [1]:
import base64
import pandas as pd
import requests
import requests_negotiate_sspi
from requests_negotiate_sspi import HttpNegotiateAuth
from pandas.io.json import json_normalize
#import arcpy
import os
from datetime import datetime
#from arcgis.features import GeoAccessor,GeoSeriesAccessor
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option("display.max_colwidth",None)

### Setting up variables and requesting a response from the Entities GET function. Then converting the entityTypeList array into a pandas dataframe. The entity TypeList array is a list of dictionaries with key-value pairs. It has entityType,name,description,count,and URL. I take this information and export to excel. This spreadsheet will give us a better understanding of all the entity types we can call within the Studio API. 

In [2]:
# Getting all entity types and descriptions from Studio API
base_url = "http://hoewap609:50231"
connection = "P_PET_SAMP"
repository = "GUYANA_INT"
pagecount = 400

url = f"{base_url}/api/studio/petrel/entities/{connection}/{repository}?entitiesPerPage={pagecount}"
entityTypes = requests.get(url, auth=HttpNegotiateAuth())
entityTypes.raise_for_status()
print(entityTypes.json()) #Response in json format

entityTypes_df = pd.json_normalize(entityTypes.json()['entityTypeList']) #normalizing json formation with pandas json normalize function. This will convert keys to columns and values to data within cells.
entityTypes_df.to_excel(r"H:\Studio_ApI_entityTypes.xlsx") # to excel file
display(entityTypes_df) #displaying dataframe

{'pageNumber': 0, 'entityCount': 237, 'entitiesPerPage': 400, 'pageCount': 1, 'firstPageURL': 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT?page=0&entitiesPerPage=400', 'previousPageURL': None, 'nextPageURL': None, 'lastPageURL': 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT?page=0&entitiesPerPage=400', 'entityTypeList': [{'entityType': 'DbxCoordinateSystem', 'name': 'SRD', 'description': 'Reference coordinate system associated with lateral and vertical measurements', 'count': 41698, 'entityListURL': 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxCoordinateSystem'}, {'entityType': 'DbxReplacementVelocity', 'name': 'Replacement velocity', 'description': 'A place to store Replacement Velocity information. This is assumed velocity between From_SRD and To_SRD.', 'count': 0, 'entityListURL': 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxReplacementVelocity'}, {'entityType': 'DbxH

,entityType,name,description,count,entityListURL
0,DbxCoordinateSystem,SRD,Reference coordinate system associated with lateral and vertical measurements,41698,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxCoordinateSystem
1,DbxReplacementVelocity,Replacement velocity,A place to store Replacement Velocity information. This is assumed velocity between From_SRD and To_SRD.,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxReplacementVelocity
2,DbxHistoryEntry,,History entry associated with Petrel domain object,378813,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxHistoryEntry
3,DbxCustomAttribute,None,A non-Dbx custom attribute for a DbxObject,5268,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxCustomAttribute
4,DbxPetrelProject,,Petrel Project Information,5140,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPetrelProject
5,DbxPetrelObject,,Petrel Object Information,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPetrelObject
6,DbxManagedProjectGroup,None,Group Managed Petrel Project,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxManagedProjectGroup
7,DbxManagedProject,None,Managed Petrel Project,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxManagedProject
8,DbxManagedObject,Managed object,Petrel Object Information,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxManagedObject
9,DbxObjectUse,None,Association between Objects,0,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxObjectUse


#### I noticed the polygons within studio are stored in the entitye DbxPolylineset. We will make a call to the API to see what metadata is avialable to extract. I then convert the entityList array into a dataframe that create a frame that consists of each polygon guid, name, and entityURL. 

In [3]:
entityType = "DbxPointWellLog" # Polygons in studio, A set of polylines.
pagecount = 2000

pointwell_url = f"{base_url}/api/studio/petrel/entities/{connection}/{repository}/{entityType}?entitiesPerPage={pagecount}"
pointwell_req = requests.get(pointwell_url, auth=HttpNegotiateAuth())
pointwell_req.raise_for_status()

df_point = pd.json_normalize(pointwell_req.json()["entityList"])
print(len(df_point))
display(df_point)


129


,guid,name,entityURL
0,010AE216BB9F4D2FAA9D2F86F8113398,Final_RCAL,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/010AE216BB9F4D2FAA9D2F86F8113398
1,028767F155E7464CB668C8D5D593AA34,SURINAME CASING POINTS,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/028767F155E7464CB668C8D5D593AA34
2,02A1150FDCFF41278BA6A4263AB785AC,Formation_Pressure,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/02A1150FDCFF41278BA6A4263AB785AC
3,046F15FE21E14EE5A2E1A729D5CF4C96,Formation_Pressure,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/046F15FE21E14EE5A2E1A729D5CF4C96
4,0487F7422C374F0CB903D54EFD2BFDE9,SANM_SRGDB_QC,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/0487F7422C374F0CB903D54EFD2BFDE9
5,0719C421A02545DB970371FED7B43FC8,Formation_Pressure,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/0719C421A02545DB970371FED7B43FC8
6,0778494AE3A445B6B4C44579AE9CCDA5,Final_RCAL,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/0778494AE3A445B6B4C44579AE9CCDA5
7,08AC0A89749B4063BCF4132B5F0DA2D2,Final_RCAL,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/08AC0A89749B4063BCF4132B5F0DA2D2
8,0907E3A0E64B42D9B3A746EC6C248FCD,Formation_Pressure,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/0907E3A0E64B42D9B3A746EC6C248FCD
9,0FB09CFAED7F43AA8BA98148635FD3A2,Formation_Pressure,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/0FB09CFAED7F43AA8BA98148635FD3A2


The script below iterates through the polygon dataframe, calls the polylineset url, calls the object url, checks if the folder path from the objects url is equal to the live trace outline path. If it is, it creates a polyline from that. If not, it skips it. Script works, but I need to manipulate the name of the polygon because the polygon names have invalid characters that Arc does not like. "#" as leading character, "-","()","."," " etc...

In [5]:
FinalRcal = "http://hoewap609:50231/api/studio/petrel/objects/P_PET_SAMP/GUYANA_INT/DbxPointWellLog/010AE216BB9F4D2FAA9D2F86F8113398"
finalrcal_req = requests.get(FinalRcal,auth = HttpNegotiateAuth())
finalrcal_req.raise_for_status()

df_rcal = pd.json_normalize(finalrcal_req.json())
display(df_rcal)

a = finalrcal_req.json()["PointWellLogProperties"]
decodedbytes = base64.b64decode(a)
decodedstr = str(decodedbytes,"utf-8")
#print(decodedstr)

properties.Id                   properties.Guid properties.Name  \
0       33598896  010AE216BB9F4D2FAA9D2F86F8113398      Final_RCAL   

  properties.JsonDoc properties.JsonAdminDoc properties.BoreholeName  \
0               None                    None                  Liza-2   

  properties.UWI  properties.DepthMin  properties.DepthMax  \
0    21418000420               5329.1              5386.26   

  properties.Comments  properties.PreferredFlag  properties.DatumElevation  \
0                None                      True                       31.6   

  properties.DepthProperty properties.TimeProperty  properties.BulkArrayId  \
0           Measured_Depth            Two_Way_Time                33598875   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

KeyError: 'PointWellLogProperties'

In [5]:
entityType = "DbxPointWellLogPropertyType" # Polygons in studio, A set of polylines.
pagecount = 2000

pointwell_url = f"{base_url}/api/studio/petrel/entities/{connection}/{repository}/{entityType}?entitiesPerPage={pagecount}"
pointwell_req = requests.get(pointwell_url, auth=HttpNegotiateAuth())
pointwell_req.raise_for_status()

df_point = pd.json_normalize(pointwell_req.json()["entityList"])
print(len(df_point))
display(df_point)

132


,guid,name,entityURL
0,02ECCB3AA5834CEEAF27367F819A840E,TOC Wt%,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/02ECCB3AA5834CEEAF27367F819A840E
1,05D14E13C960411D9569C65BF2D100DC,Start_Numeric_Age,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/05D14E13C960411D9569C65BF2D100DC
2,07D9A375D3924FE4BEF7D9ED614F9ACE,Unique_ID,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/07D9A375D3924FE4BEF7D9ED614F9ACE
3,08D0304A400141D090909FE15FCEFAE6,Depth (sub mud),http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/08D0304A400141D090909FE15FCEFAE6
4,0998F2E15FF840D6842AEE14E17D207F,Date Collected,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/0998F2E15FF840D6842AEE14E17D207F
5,09CC201EE6D84CB1A0F5758670DE619D,Rep HI min Calc HI,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/09CC201EE6D84CB1A0F5758670DE619D
6,0B22150216484194BB335E9B3605696A,Shallow_Depth_meters,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/0B22150216484194BB335E9B3605696A
7,0CD5BE2D97F34DEBA8BD8A62B2007D9E,Calculated PI,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/0CD5BE2D97F34DEBA8BD8A62B2007D9E
8,0DB135E84716486F8706DB0AD5C0CD3B,Porosity,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/0DB135E84716486F8706DB0AD5C0CD3B
9,10685E6812104D0D8C0A021597477ABD,SCAL Pc-Porosity,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/10685E6812104D0D8C0A021597477ABD


In [17]:
FinalRcal = "http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogPropertyType/71D1168DB5A34B6D8408725867B69B5A"
finalrcal_req = requests.get(FinalRcal,auth = HttpNegotiateAuth())
finalrcal_req.raise_for_status()

df_rcal = pd.json_normalize(finalrcal_req.json())
df_rcal.transpose()

,0
Id,33598824
Guid,71D1168DB5A34B6D8408725867B69B5A
CreateDate,2021-04-13T10:56:20
CreateUser,JIAVILL
UpdateDate,2021-04-13T10:56:20
UpdateUser,JIAVILL
JsonDoc,None
JsonAdminDoc,None
Name,Permeability mD
Comments,None


In [22]:
FinalRcal = "http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/71D1168DB5A34B6D8408725867B69B5A"
finalrcal_req = requests.get(FinalRcal,auth = HttpNegotiateAuth())
finalrcal_req.raise_for_status()

df_rcal = pd.json_normalize(finalrcal_req.json())
df_rcal.transpose()

HTTPError: 500 Server Error: Internal Server Error for url: http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/71D1168DB5A34B6D8408725867B69B5A

In [18]:
FinalRcal = "http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogVersion/0AB949227D1341E2884FE01E965AB82A"
finalrcal_req = requests.get(FinalRcal,auth = HttpNegotiateAuth())
finalrcal_req.raise_for_status()

df_rcal = pd.json_normalize(finalrcal_req.json())
df_rcal.transpose()

,0
Id,33598820
Guid,0AB949227D1341E2884FE01E965AB82A
Name,Final_RCAL
JsonDoc,None
JsonAdminDoc,None
Comments,Data Source: WIRE\r\n
SemName,Final_RCAL
SemRemarks,Data Source: WIRE\r\n
LogVersionCollectionURL,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxLogVersionCollection/07B65512D7234FCE8833731B6A61A5DA
dbxObjectURL,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxObject/0AB949227D1341E2884FE01E965AB82A


In [20]:
FinalRcal = "http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxObject/0AB949227D1341E2884FE01E965AB82A"
finalrcal_req = requests.get(FinalRcal,auth = HttpNegotiateAuth())
finalrcal_req.raise_for_status()

df_rcal = pd.json_normalize(finalrcal_req.json())
df_rcal.transpose()

,0
Id,33598820
EntityType,DbxPointWellLogVersion
EntityTable,DovPointWellLogVersion
EntityId,33598820
DisplayEntityType,Point well data
DisplayGenericEntityType,Point well data
GenericEntityType,DbxPointWellLogVersion
FolderPath,\Project\Input\Wells\Global well logs\CORE
FolderPathType,\DbxProjectGroup\DbxTreeRootGroup\DbxBoreholeCollection\DbxLogVersionCollection\DbxLogVersionCollection
PresentationParentGuid,07B65512D7234FCE8833731B6A61A5DA


In [6]:
entityType = "DbxPointWellLogVersion" # Polygons in studio, A set of polylines.
pagecount = 2000

pointwell_url = f"{base_url}/api/studio/petrel/entities/{connection}/{repository}/{entityType}?entitiesPerPage={pagecount}"
pointwell_req = requests.get(pointwell_url, auth=HttpNegotiateAuth())
pointwell_req.raise_for_status()

df_point = pd.json_normalize(pointwell_req.json()["entityList"])
print(len(df_point))
display(df_point)

7


,guid,name,entityURL
0,0AB949227D1341E2884FE01E965AB82A,Final_RCAL,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogVersion/0AB949227D1341E2884FE01E965AB82A
1,4C302ABC41604B3BAE3A67FB5251BFD2,Rock_data_Presalt,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogVersion/4C302ABC41604B3BAE3A67FB5251BFD2
2,6C76A5F388E64DD3AC51DBD849E63BB4,SURINAME CASING POINTS,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogVersion/6C76A5F388E64DD3AC51DBD849E63BB4
3,9B0625E9FC364F6D834372883FBEE863,Formation_Pressure,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogVersion/9B0625E9FC364F6D834372883FBEE863
4,ECC9CB28F5FC4310B139FF0145CFFF03,Rock_descr_Presalt,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogVersion/ECC9CB28F5FC4310B139FF0145CFFF03
5,F3DD541383934C9681B971DAA5760FB4,SANM_SRGDB,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogVersion/F3DD541383934C9681B971DAA5760FB4
6,FEF11ABB00654F518EB378059B0C56A8,SANM_SRGDB_QC,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogVersion/FEF11ABB00654F518EB378059B0C56A8


In [8]:
FinalRcal = "http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxPointWellLogVersion/0AB949227D1341E2884FE01E965AB82A"
finalrcal_req = requests.get(FinalRcal,auth = HttpNegotiateAuth())
finalrcal_req.raise_for_status()

df_rcal = pd.json_normalize(finalrcal_req.json())
df_rcal.transpose()

,0
Id,33598820
Guid,0AB949227D1341E2884FE01E965AB82A
Name,Final_RCAL
JsonDoc,None
JsonAdminDoc,None
Comments,Data Source: WIRE\r\n
SemName,Final_RCAL
SemRemarks,Data Source: WIRE\r\n
LogVersionCollectionURL,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxLogVersionCollection/07B65512D7234FCE8833731B6A61A5DA
dbxObjectURL,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxObject/0AB949227D1341E2884FE01E965AB82A


In [9]:
logversioncollection = "http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxLogVersionCollection/07B65512D7234FCE8833731B6A61A5DA"
finalrcal_req = requests.get(logversioncollection,auth = HttpNegotiateAuth())
finalrcal_req.raise_for_status()

df_rcal = pd.json_normalize(finalrcal_req.json())
df_rcal.transpose()

,0
Id,912596
Guid,07B65512D7234FCE8833731B6A61A5DA
Name,CORE
JsonDoc,None
JsonAdminDoc,None
Comments,"Continuous log curves given the LOG NAME or LOG TYPE of ""CORE"" in Recall These are not point well data or discrete curves."
RootBoreholeCollection,None
ParentLogVersionCollectionURL,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxLogVersionCollection/B40B4AEB666A4E079626029978EFC179
dbxObjectURL,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxObject/07B65512D7234FCE8833731B6A61A5DA
ObjectsURL,None


what does some polygons not have the ["LineStrings3"] key?

In [10]:
dbxobjecturl = "http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxObject/0AB949227D1341E2884FE01E965AB82A"
finalrcal_req = requests.get(dbxobjecturl,auth = HttpNegotiateAuth())
finalrcal_req.raise_for_status()

df_rcal = pd.json_normalize(finalrcal_req.json())
df_rcal.transpose()

,0
Id,33598820
EntityType,DbxPointWellLogVersion
EntityTable,DovPointWellLogVersion
EntityId,33598820
DisplayEntityType,Point well data
DisplayGenericEntityType,Point well data
GenericEntityType,DbxPointWellLogVersion
FolderPath,\Project\Input\Wells\Global well logs\CORE
FolderPathType,\DbxProjectGroup\DbxTreeRootGroup\DbxBoreholeCollection\DbxLogVersionCollection\DbxLogVersionCollection
PresentationParentGuid,07B65512D7234FCE8833731B6A61A5DA


In [15]:
entityType = "DbxWellLogPropertyValue" # Polygons in studio, A set of polylines.
pagecount = 2000

pointwell_url = f"{base_url}/api/studio/petrel/entities/{connection}/{repository}/{entityType}?entitiesPerPage={pagecount}"
pointwell_req = requests.get(pointwell_url, auth=HttpNegotiateAuth())
pointwell_req.raise_for_status()

df_point = pd.json_normalize(pointwell_req.json()["entityList"])
print(len(df_point))
display(df_point)

2000


,guid,name,entityURL
0,0002DF3EA10C46FFBA2841E8B5C797A4|Run date,,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAwMkRGM0VBMTBDNDZGRkJBMjg0MUU4QjVDNzk3QTR8UnVuIGRhdGU%3D
1,0008EC65A93B488A9958B6F6F2BF6ED8|Log version,,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAwOEVDNjVBOTNCNDg4QTk5NThCNkY2RjJCRjZFRDh8TG9nIHZlcnNpb24%3D
2,0008EC65A93B488A9958B6F6F2BF6ED8|Run date,,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAwOEVDNjVBOTNCNDg4QTk5NThCNkY2RjJCRjZFRDh8UnVuIGRhdGU%3D
3,0012228BDF8D48E4AEC397428EAF50FA|Log version,,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAxMjIyOEJERjhENDhFNEFFQzM5NzQyOEVBRjUwRkF8TG9nIHZlcnNpb24%3D
4,0012228BDF8D48E4AEC397428EAF50FA|Run date,,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAxMjIyOEJERjhENDhFNEFFQzM5NzQyOEVBRjUwRkF8UnVuIGRhdGU%3D
5,0021770183F94B52AB1081F44575C2FB|Run date,,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAyMTc3MDE4M0Y5NEI1MkFCMTA4MUY0NDU3NUMyRkJ8UnVuIGRhdGU%3D
6,0022160AC5A14A7BB1183E0F679294AF|Log version,,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAyMjE2MEFDNUExNEE3QkIxMTgzRTBGNjc5Mjk0QUZ8TG9nIHZlcnNpb24%3D
7,0022160AC5A14A7BB1183E0F679294AF|Run date,,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAyMjE2MEFDNUExNEE3QkIxMTgzRTBGNjc5Mjk0QUZ8UnVuIGRhdGU%3D
8,0031C1E857FC4B90A073852673BBDCD3|Log version,,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAzMUMxRTg1N0ZDNEI5MEEwNzM4NTI2NzNCQkRDRDN8TG9nIHZlcnNpb24%3D
9,0031C1E857FC4B90A073852673BBDCD3|Run date,,http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAzMUMxRTg1N0ZDNEI5MEEwNzM4NTI2NzNCQkRDRDN8UnVuIGRhdGU%3D


In [16]:
url = 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyValue/b64_MDAwMkRGM0VBMTBDNDZGRkJBMjg0MUU4QjVDNzk3QTR8UnVuIGRhdGU%3D'
req = requests.get(url,auth=HttpNegotiateAuth())
req.json()

{'Id': 7893556, 'Guid': '0002DF3EA10C46FFBA2841E8B5C797A4|Run date', 'CreateDate': '2016-08-16T15:58:10', 'CreateUser': 'AWHANDY', 'UpdateDate': '2016-08-16T15:58:10', 'UpdateUser': 'AWHANDY', 'JsonDoc': None, 'JsonAdminDoc': None, 'DataType': 'Date', 'BooleanValue': False, 'DateValue': '0001-01-01T00:00:00', 'StringValue': None, 'FloatValue': 'NaN', 'FloatValueUom': None, 'IntegerValue': -2147483648, 'IntegerValueUom': None, 'ParentObject': {'entityType': 'DbxGenericWellLog', 'uniqueId': '0002DF3EA10C46FFBA2841E8B5C797A4'}, 'PropertyType': {'entityType': 'DbxWellLogPropertyType', 'uniqueId': 'Run date'}, 'ParentObjectURL': 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxGenericWellLog/0002DF3EA10C46FFBA2841E8B5C797A4', 'PropertyTypeURL': 'http://hoewap609:50231/api/studio/petrel/entities/P_PET_SAMP/GUYANA_INT/DbxWellLogPropertyType/b64_UnVuIGRhdGU%3D'}